
# This notebook has Zero shot classifier Implementation with both

# 1.   Server Side Implementation
**Implemention done in Flask and Huggingface Transformers Library.**

# 2.   Client Side Implementation
**Implementation done using Python requests Package.**



In [ ]:
! pip install transformers

In [2]:
import pandas as pd
from transformers import pipeline
from IPython.core.display import HTML
import socket
from flask import Flask, jsonify, request
print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

172.28.0.2


In [3]:
classifier = pipeline("zero-shot-classification")

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Server Side Implementation

### Flask Post end-point

In [4]:
app = Flask(__name__)

@app.route("/")
def Hello():
    return "Hello World...."

@app.route('/classifier', methods=['POST'])                          # Implementation of Classifier.
def JsonMethod():
    content = request.get_json()

    try:
      query_text=(content["sentence"])
      query_category=(content["category"])
    except:
      return jsonify(str("Data is missing." ))



    if (type(query_category) is list) and (type(query_text) is str):
      try:
        result = classifier(query_text, query_category)
        prediction=list(zip(result["labels"], result["scores"]))
        return jsonify(str(prediction))                                      # Return Tuple of category with probability.
      except:
        return jsonify(str("Data is not correctly formatted. Text should be 'str', while category should be 'list array'. " ))

    
    else:
        return jsonify(str("Datatype mismatched, text should be str, while category should be array" ))



## Running Server instance on a thread. This way, we can still run client in the same notebook.

In [5]:
import threading
t1=threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':80})

In [6]:
t1.start()

 * Serving Flask app "__main__" (lazy loading)


In [7]:
t1.is_alive()

True

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)


# Client Side Implementation
## Python requests package is used to, talk to the server.

In [9]:
import requests

data = {
        'sentence':   "I complained to the respective department under federal, and they solved my issue in a day. Impressed",
        'category': ["Police",
                     "poor governance",
                     "App not working",
                     "amazing app",
                     "good governance"],
        }

res = requests.post('http://172.28.0.2/classifier', json=data) # Connecting to Server, using Python requests package, and sending Post request.

if res.ok:
    print(res.json())

172.28.0.2 - - [29/Dec/2020 04:46:11] "POST /classifier HTTP/1.1" 200 -


[('good governance', 0.6999590992927551), ('Police', 0.13267572224140167), ('amazing app', 0.0907571092247963), ('App not working', 0.06189846247434616), ('poor governance', 0.014709617011249065)]


In [10]:
import requests
data = {
        'sentence':   "I complained to the respective department under federal, but they did nothing to my complain",
        'category': ["Police",
                     "poor governance",
                     "App not working",
                     "amazing app",
                     "good governance"],
        }

res = requests.post('http://172.28.0.2/classifier', json=data)  # Connecting to Server, using Python requests package, and sending Post request.

if res.ok:
    print(res.json())


172.28.0.2 - - [29/Dec/2020 04:46:17] "POST /classifier HTTP/1.1" 200 -


[('poor governance', 0.7187206745147705), ('App not working', 0.14282187819480896), ('Police', 0.10261546820402145), ('good governance', 0.024068979546427727), ('amazing app', 0.01177296508103609)]
